In [ ]:
# 実行には，spaCy3.X and GiNZA 5.Xが必要です。venv等で仮想環境を構築することをおすすめします．
# !python -m venv ginza5X
# !source ginza5X/bin/activate
# !pip install ja-ginza-electra requests pandas jupyterlab ipywidgets openpyxl
# !jupyter-lab

In [ ]:
import os
import time

import pandas as pd
import requests
import spacy

## モデルのロード
nlp = spacy.load("ja_ginza_electra")
data_dir = "sisyou_db"

In [ ]:
# 1.労災データファイルのダウンロード
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

years = [28, 29]
months = list(range(1, 13))

for y in years:
    for m in months:
        file_name = f"{data_dir}/sisyou_db_h{y}_{m:02d}.xlsx"
        print(file_name)
        if not os.path.exists(file_name):
            file_url = f"https://anzeninfo.mhlw.go.jp/anzen/shisyo_xls/sisyou_db_h{y}_{m:02d}.xlsx"
            res = requests.get(file_url)
            with open(file_name, 'wb') as f:
                f.write(res.content)
            time.sleep(2)    

In [ ]:
# 2. 分類対象データの読み込み
## 労災データの読み込み
def read_rousai_db(data_dir, year_months):
    dfs = []
    for y, m in year_months:
        file_name = f"{data_dir}/sisyou_db_h{y}_{m:02d}.xlsx"
        df = pd.read_excel(file_name, skiprows=[1])
        dfs.append(df)

    df_ret = pd.concat(dfs)
    df_ret = df_ret.rename(columns={"事故の型": "事故の型_コード",
                                    "Unnamed: 20": "事故の型_名前"})
    return df_ret

## 訓練データ・テストデータの読み込み
year_months_train = [(28, m) for m in months]
year_months_dev = [(29, 1)]
year_months_test = [(29, 2)]
df_train = read_rousai_db(data_dir, year_months_train)
df_dev = read_rousai_db(data_dir, year_months_dev)
df_test = read_rousai_db(data_dir, year_months_test)

In [ ]:
# 3.分類クラスを頻度上位10クラス+それ以外の11クラスに絞り込み
names = df_train["事故の型_名前"].unique()
top10_category = df_train["事故の型_名前"].value_counts()[0:10].index.to_list()
name2label = {name: top10_category.index(name) if name in top10_category else len(top10_category) for i, name in enumerate(names)}
print(top10_category)


In [ ]:
# 4.spaCy Command Line Interface 用の訓練データ・テストデータのファイルを生成
from spacy.tokens import DocBin

def convert_docs(df_target, dic_label):
    list_docs = []
    for index, entry in df_target.iterrows():
        doc = nlp.make_doc(entry["災害状況"])
        for label,val in dic_label.items():
            if(label == entry["事故の型_名前"]):
                doc.cats[str(val)] = 1
            else:
                doc.cats[str(val)] = 0

        list_docs.append(doc)
    return list_docs

docs_train = convert_docs(df_train, name2label)
doc_bin_train = DocBin(docs=docs_train)
doc_bin_train.to_disk("./train_ja.spacy")

docs_dev = convert_docs(df_dev, name2label)
doc_bin_dev = DocBin(docs=docs_dev)
doc_bin_dev.to_disk("./dev_ja.spacy")

In [ ]:
# 5.分類器の学習を実行 => 文書分類モデルを./rousai_classifier　に保存
## configファイルの生成
!python -m spacy init fill-config ./base_ginza_textcat.cfg ./ginza_textcat.cfg
## 訓練の実行
!python -m spacy train ./ginza_textcat.cfg --output ./rousai_classifier

In [ ]:
# 6.性能がベストだった文書分類モデルを読み込み、評価用データに適用
nlp_best = spacy.load("rousai_classifier/model-best")
counter = 0
prec = 0
for index, entry in df_test.iterrows():
    doc = nlp_best(entry["災害状況"])
    label = entry["事故の型_名前"]

    ## 1番スコアの高いクラスを予測値とする
    for k,v in sorted(doc.cats.items(),key=lambda x:x[1],reverse=True):
        toprank = int(k)
        break
    ## 評価用データのラベルを分類クラスIDに変換
    answer = name2label[label]

    if toprank == answer:
        prec += 1
    counter += 1

print("{}({}/{})".format(prec/counter,prec,counter))